In [48]:
import pandas as pd
df = pd.read_csv(r"C:\Users\LENOVO\OneDrive\Desktop\downloads\Banglore Housing Prices.csv", encoding='ISO-8859-1')


In [49]:
df.describe() 

,bath,price
count,13247.000000,13320.000000
mean,2.692610,112.565627
std,1.341458,148.971674
min,1.000000,8.000000
25%,2.000000,50.000000
50%,2.000000,72.000000
75%,3.000000,120.000000
max,40.000000,3600.000000


In [50]:

print(df.shape)  # should show number of rows > 0


(13320, 5)


In [5]:
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [6]:
df.columns

Index(['location', 'size', 'total_sqft', 'bath', 'price'], dtype='object')

In [24]:
df.isnull().sum()                     # Count missing values per column
           


location          0
size              0
total_sqft        0
bath              0
price             0
Price_Per_Sqft    0
dtype: int64

In [51]:
df = df.dropna(subset=['size', 'total_sqft', 'bath', 'price'])


In [52]:
print(df.head())                   # Preview first few rows
print(df.info())  


                   location       size total_sqft  bath   price
0  Electronic City Phase II      2 BHK       1056   2.0   39.07
1          Chikka Tirupathi  4 Bedroom       2600   5.0  120.00
2               Uttarahalli      3 BHK       1440   2.0   62.00
3        Lingadheeranahalli      3 BHK       1521   3.0   95.00
4                  Kothanur      2 BHK       1200   2.0   51.00
<class 'pandas.core.frame.DataFrame'>
Index: 13247 entries, 0 to 13319
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    13246 non-null  object 
 1   size        13247 non-null  object 
 2   total_sqft  13247 non-null  object 
 3   bath        13247 non-null  float64
 4   price       13247 non-null  float64
dtypes: float64(2), object(3)
memory usage: 621.0+ KB
None


In [53]:
df['size'] = df['size'].astype(str).str.extract('(\d+)').astype(float)


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_19912\2508173456.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['size'] = df['size'].astype(str).str.extract('(\d+)').astype(float)


In [54]:
df['size'] = df['size'].astype(int)

In [60]:
df['total_sqft'] = df['total_sqft'].apply(lambda x: (float(x.split('-')[0]) + float(x.split('-')[1])) / 2 if '-' in x else pd.to_numeric(x, errors='coerce'))
df.dropna(subset=['total_sqft'], inplace=True)


In [61]:
df['Price_Per_Sqft'] = df['price'] / df['total_sqft']  # Price per square foot


In [41]:
df['total_sqft'] = df['total_sqft'].apply(lambda x: sum(map(float, str(x).split('-'))) / 2 if '-' in str(x) else float(x))


In [62]:
# Removing outliers using Z-score method
!pip install scipy

from scipy.stats import zscore
df = df[(zscore(df['Price_Per_Sqft']) < 3) & (zscore(df['size']) < 3)]  # Z-score > 3 are outliers


Defaulting to user installation because normal site-packages is not writeable


In [63]:
!pip install scikit-learn



Defaulting to user installation because normal site-packages is not writeable


In [64]:
print(df.shape)
print(df[['size', 'total_sqft', 'bath', 'price']].isnull().sum())

(12931, 6)
size          0
total_sqft    0
bath          0
price         0
dtype: int64


In [65]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Feature selection
X = df[['size', 'total_sqft', 'bath']]
y = df['price']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Performance metrics
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Output results
print(f"Training MSE: {mse_train:.2f}")
print(f"Testing MSE: {mse_test:.2f}")
print(f"Training R² Score (Accuracy): {r2_train:.2f}")
print(f"Testing R² Score (Accuracy): {r2_test:.2f}")


Training MSE: 11652.27
Testing MSE: 10968.00
Training R² Score (Accuracy): 0.40
Testing R² Score (Accuracy): 0.51
